In [57]:
from openai import OpenAI
from transformers import AutoTokenizer
import requests
import httpx
import numpy as np

In [18]:
client = OpenAI(base_url="http://localhost:8000/v1", api_key="-")
client = OpenAI(base_url="http://64.181.228.72:8000/v1", api_key="-")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Math-PRM-7B")
tokenizer

Qwen2TokenizerFast(name_or_path='Qwen/Qwen2.5-Math-PRM-7B', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<R>', '<S>', '<X>', '<mask>', '<sep>', '<extra_0>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<R>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151647: AddedToken("<S>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151648: AddedToken("<X>", 

In [54]:

data = {
    "system": "Please reason step by step, and put your final answer within \\boxed{}.",
    "query": "Sue lives in a fun neighborhood.  One weekend, the neighbors decided to play a prank on Sue.  On Friday morning, the neighbors placed 18 pink plastic flamingos out on Sue's front yard.  On Saturday morning, the neighbors took back one third of the flamingos, painted them white, and put these newly painted white flamingos back out on Sue's front yard.  Then, on Sunday morning, they added another 18 pink plastic flamingos to the collection. At noon on Sunday, how many more pink plastic flamingos were out than white plastic flamingos?",
    "response": [
      "To find out how many more pink plastic flamingos were out than white plastic flamingos at noon on Sunday, we can break down the problem into steps. First, on Friday, the neighbors start with 18 pink plastic flamingos.",
      "On Saturday, they take back one third of the flamingos. Since there were 18 flamingos, (1/3 \\times 18 = 6) flamingos are taken back. So, they have (18 - 6 = 12) flamingos left in their possession. Then, they paint these 6 flamingos white and put them back out on Sue's front yard. Now, Sue has the original 12 pink flamingos plus the 6 new white ones. Thus, by the end of Saturday, Sue has (12 + 6 = 18) pink flamingos and 6 white flamingos.",
      "On Sunday, the neighbors add another 18 pink plastic flamingos to Sue's front yard. By the end of Sunday morning, Sue has (18 + 18 = 36) pink flamingos and still 6 white flamingos.",
    #   "To find the difference, subtract the number of white flamingos from the number of pink flamingos: (36 - 6 = 30). Therefore, at noon on Sunday, there were 30 more pink plastic flamingos out than white plastic flamingos. The answer is (\\boxed{30})."
      "To find the difference, add the number of white flamingos from the number of pink flamingos: (36 + 6 = 42). Therefore, at noon on Sunday, there were 42 more pink plastic flamingos out than white plastic flamingos. The answer is (\\boxed{42})."
    ]
}

messages = [
    {"role": "system", "content": data['system']},
    {"role": "user", "content": data['query']},
    {"role": "assistant", "content": "<extra_0>".join(data['response']) + "<extra_0>"},
]
conversation_str = tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=False
)
conversation_str

"<|im_start|>system\nPlease reason step by step, and put your final answer within \\boxed{}.<|im_end|>\n<|im_start|>user\nSue lives in a fun neighborhood.  One weekend, the neighbors decided to play a prank on Sue.  On Friday morning, the neighbors placed 18 pink plastic flamingos out on Sue's front yard.  On Saturday morning, the neighbors took back one third of the flamingos, painted them white, and put these newly painted white flamingos back out on Sue's front yard.  Then, on Sunday morning, they added another 18 pink plastic flamingos to the collection. At noon on Sunday, how many more pink plastic flamingos were out than white plastic flamingos?<|im_end|>\n<|im_start|>assistant\nTo find out how many more pink plastic flamingos were out than white plastic flamingos at noon on Sunday, we can break down the problem into steps. First, on Friday, the neighbors start with 18 pink plastic flamingos.<extra_0>On Saturday, they take back one third of the flamingos. Since there were 18 flam

In [23]:
client.completions.create(
    model="Qwen/Qwen2.5-Math-PRM-7B",
    prompt=conversation_str,
)

Completion(id=None, choices=None, created=None, model=None, object='error', system_fingerprint=None, usage=None, message='The model does not support Completions API', type='BadRequestError', param=None, code=400)

In [ ]:
async def fetch_pooling(conversation_str: str) -> list[float]:
    data = {
        "model": "Qwen/Qwen2.5-Math-PRM-7B",
        "input": conversation_str,
    }
    async with httpx.AsyncClient() as client:
        r = await client.post("http://64.181.228.72:8000/pooling", json=data)
        result = r.json()
    process_rewards = [a[1] for a in result["data"][0]["data"]]
    return process_rewards


await fetch_pooling(conversation_str)


[1.0, 0.1552734375, 0.97265625, 0.0047607421875]

In [ ]:
data = {
    "model": "Qwen/Qwen2.5-Math-PRM-7B",
    "input": conversation_str,
}
r = requests.post("http://64.181.228.72:8000/pooling",json=data)
r.json()

{'id': 'pool-c18cb8e6c2534e6e8ed95d0132690726',
 'object': 'list',
 'created': 1742913429,
 'model': 'Qwen/Qwen2.5-Math-PRM-7B',
 'data': [{'index': 0,
   'object': 'pooling',
   'data': [[0.0005035400390625, 1.0],
    [0.84375, 0.1552734375],
    [0.0255126953125, 0.97265625],
    [0.99609375, 0.0047607421875]]}],
 'usage': {'prompt_tokens': 454,
  'total_tokens': 454,
  'completion_tokens': 0,
  'prompt_tokens_details': None}}

In [ ]:
 # [[1.0, 0.1904296875, 0.9765625, 1.0]]